In [2]:
import pandas as pd
import numpy as np
from Bio.Data.IUPACData import protein_letters_3to1
import re
import os
import db_utils
import sqlalchemy
from find_path import find_file
from collections import OrderedDict
import warnings

In [ ]:
# generate the input for uniprot mapping from clinvar:
# refseq = clinvar['RefSeq_ID'].to_frame()
# refseq.drop_duplicates(inplace=True)
# refseq.to_csv('refseqIDs.txt', sep='\n', header= False, index = False)

# in uniprot, I used from RefSeq Nucleotide to UniProtKB mapping and just added one column from external resources and then genome annotation: ensebml, from where I extracted the uniprot isoform information

# get the output
# mapping = pd.read_csv('RefSeqToUniProt.tsv', sep='\t')
# mapping.rename(columns={'From':'RefSeq_ID', 'Entry':'uni_AC'}, inplace=True)

# # extract isoform ID from the ensembl column
# pattern = r'\[([^\]]+)\]'
# matches = mapping['Ensembl'].str.extractall(pattern)
# result = matches.groupby(level=0)[0].apply(list)
# mapping['Ref_Seq_to_iso'] = result
# mapping = mapping[['RefSeq_ID','uni_AC','Ref_Seq_to_iso']]
# mapping = mapping.explode('Ref_Seq_to_iso')
# mapping.drop_duplicates(inplace=True)

In [5]:
clinvar

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
0,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,NaN,397704705,...,"criteria provided, single submitter",2,NaN,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4820844,GGAT,TGCTGTAAACTGTAACTGTAAA
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,NaN,397704705,...,"criteria provided, single submitter",2,NaN,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA
2,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,no assertion criteria provided,1,NaN,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4827360,GCTGCTGGACCTGCC,G
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,no assertion criteria provided,1,NaN,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4787729,GCTGCTGGACCTGCC,G
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,no assertion criteria provided,1,NaN,N,"ClinGen:CA210674,UniProtKB:Q92610#VAR_064583,O...",1,4,85342440,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4780028,2831556,single nucleotide variant,NM_170707.4(LMNA):c.1685T>C (p.Leu562Pro),4000,LMNA,HGNC:6636,Uncertain significance,0,"Oct 16, 2023",-1,...,"criteria provided, single submitter",1,"ACMG2013,ACMG2016,ACMG2021,ACMG2022",N,NaN,2,2664087,156107521,T,C
4780029,2831556,single nucleotide variant,NM_170707.4(LMNA):c.1685T>C (p.Leu562Pro),4000,LMNA,HGNC:6636,Uncertain significance,0,"Oct 16, 2023",-1,...,"criteria provided, single submitter",1,"ACMG2013,ACMG2016,ACMG2021,ACMG2022",N,NaN,2,2664087,156137730,T,C
4780030,2831557,Deletion,NC_000010.11:g.79055368_79303220del,57178,ZMIZ1,HGNC:16493,Pathogenic,1,"May 10, 2023",-1,...,"criteria provided, single submitter",1,NaN,N,NaN,2,2664088,-1,NaN,NaN
4780031,2831558,Deletion,NM_003042.4(SLC6A1):c.336del (p.Gly112_Leu113i...,6529,SLC6A1,HGNC:11042,Pathogenic,1,"Nov 27, 2023",-1,...,"criteria provided, single submitter",1,NaN,N,NaN,2,2664089,11059620,CG,C


In [ ]:
clinvar = pd.read_csv('ClinVar_variants.txt', sep='\t', na_values=['-','na'], low_memory=False)

In [190]:
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)

# import the new clinvar dataset:
clinvar = pd.read_csv('ClinVar_variants.txt', sep='\t', na_values=['-','na'], low_memory=False)
clinvar.rename(columns={clinvar.columns[0]:'AlleleID'}, inplace=True)
clinvar.insert(2,'RefSeq_ID', clinvar['Name'].str.split('.').str[0], True)

# reduce the dataset to GRCh38 assembly:
print('from', clinvar.shape[0])
clinvar = clinvar[clinvar['Assembly'] == 'GRCh38']
print('to', clinvar.shape[0], 'after filtering for the assembly')

# filter for single nucleotide variants
clinvar = clinvar[clinvar['Type'].str.contains('single nucleotide variant')]
print('to', clinvar.shape[0], 'after filtering for SNVs')

# and variants having the protein amino acid change in their notation
clinvar = clinvar[clinvar['Name'].str.contains('p.')]
print('to', clinvar.shape[0], 'after filtering for mutations with the p. HGVS expression')

### filter for the mutations falling into the canonical isoform:
# to map RefSeq IDs to isoform IDs, I used biomart: filtered for chromosomes, and selected the following identifiers:
# RefSeq mRNA ID
# UniProtKB isoform ID
# UniProtKB/Swiss-Prot ID
mapping = pd.read_csv('RefSeq_to_iso.txt', sep='\t')
mapping.rename(columns={'RefSeq mRNA ID':'RefSeq_ID','UniProtKB isoform ID':'Ref_Seq_to_iso', 'UniProtKB/Swiss-Prot ID':'uni_AC'}, inplace=True)
mapping.drop_duplicates(inplace=True)
mapping = mapping[~(mapping['RefSeq_ID'].isna()) & ~(mapping['uni_AC'].isna())]
mapping.loc[mapping['Ref_Seq_to_iso'].isna(), 'Ref_Seq_to_iso'] = mapping.loc[mapping['Ref_Seq_to_iso'].isna(), 'uni_AC']

# load a list of canonical isoforms:
can_iso = pd.read_csv(find_file('Canonical_isoforms.csv'), header=0, sep=',')
mapping = mapping.merge(can_iso, how='inner', left_on='uni_AC', right_on='uniprot_id')
mapping = mapping[['RefSeq_ID','uni_AC','Ref_Seq_to_iso','uniprot_iso_id']]
mapping = mapping[mapping['Ref_Seq_to_iso'] == mapping['uniprot_iso_id']][['RefSeq_ID','uni_AC','Ref_Seq_to_iso']] # only keep refseq IDs which map to canonical isoforms
mapping.drop_duplicates(inplace=True)

# restrict to mutations which map to canonical isoforms:
clinvar = clinvar.merge(mapping, how='inner', on='RefSeq_ID')
print('to', clinvar.shape[0], 'after filtering for mutations in the canonical isoforms')

# WT amino acids at the mutated site needs to match the sequence of the canonical isoform:

# extract amino acid change information:
clinvar.reset_index(inplace = True, drop = True)
# extract amino acid change
clinvar.insert(4, 'aa_change', clinvar['Name'].str.extract(r'p\.(.*?)\)')[0].tolist(), True)
# make extra column for position of aa change
clinvar.insert(5,'aa_pos', clinvar['aa_change'].str.extract(r'(\d{1,})', expand=False), True)
# make extra column to get WT amino acid -> I later checked that all the WT_aa entries consist of 3 letters.
clinvar.insert(6, 'WT_aa', clinvar['aa_change'].str.extract(r'(^[a-zA-Z]{0,3})', expand=False),True)
# convert aa into single letter
clinvar.replace({'WT_aa': protein_letters_3to1}, inplace =True)
# make extra column for alternate amino acid
# a = in the alternate amino acid column indicates that the variant has no predicted effect on the protein level
clinvar.insert(7, 'alt_aa', clinvar['aa_change'].str.extract(r'(\D{0,3}$)', expand=False),True)
# convert aa into single letter
clinvar.replace({'alt_aa': protein_letters_3to1}, inplace =True)

# merge with the sequences of uniprot isoforms
clinvar = clinvar.merge(can_iso, how='inner', left_on='Ref_Seq_to_iso', right_on='uniprot_iso_id')
# the mutation position needs to be lower than the length:
clinvar['len_ok'] = pd.Series([(int(clinvar.loc[row,'aa_pos']) <= len(clinvar.loc[row,'sequence'])) for row in clinvar.index])
clinvar = clinvar[clinvar['len_ok']]
clinvar.reset_index(inplace=True, drop=True)
print('to', clinvar.shape[0], 'after comparing with the length of the sequence of the canonical isoform')

# the WT amino acid needs to match the one we find in the same position of the canonical isoform:
clinvar['WT_aa_in_iso'] = [a[int(b)-1] for a, b in zip(clinvar['sequence'], clinvar['aa_pos'])]
clinvar = clinvar[(clinvar['WT_aa'] == clinvar['WT_aa_in_iso'])]
print('to', clinvar.shape[0], 'after restricting to mutations whose WT amino acid matches the one we find in the same position of the canonical isoform')

# let's select non-synonymous variants only:
clinvar = clinvar[(clinvar['alt_aa'] != '=')]
print('to', clinvar.shape[0], 'after taking non-synonymous mutations only')

# filter for germline mutations:
clinvar = clinvar[clinvar['OriginSimple']=='germline']
print('to', clinvar.shape[0], 'after taking germline mutations')

# filter for definite clinical significance:
clinvar = clinvar[clinvar['ClinicalSignificance'].isin(['Uncertain significance', 'Likely benign', 'Benign', 'Pathogenic', 'Likely pathogenic', 'Benign/Likely benign', 'Pathogenic/Likely pathogenic','Conflicting interpretations of pathogenicity'])]
print('to', clinvar.shape[0], 'after taking mutations with the interpretable clinical significance')





from 4780033
to 2356445 after filtering for the assembly
to 2141442 after filtering for SNVs
to 1704850 after filtering for mutations with the p. HGVS expression
to 1469843 after filtering for mutations in the canonical isoforms
to 1468891 after comparing with the length of the sequence of the canonical isoform
to 1466286 after restricting to mutations whose WT amino acid matches the one we find in the same position of the canonical isoform
to 1070494 after taking non-synonymous mutations only
to 1047477 after taking germline mutations
to 1044160 after taking mutations with the interpretable clinical significance


In [193]:
clinvar = clinvar[['AlleleID', 'VariationID', 'Chromosome', 'Start', 'ReferenceAlleleVCF','AlternateAlleleVCF', 'GeneSymbol','uni_AC', 'uniprot_iso_id', 'aa_change',  'ClinicalSignificance',  'PhenotypeList']]
clinvar.rename(columns={'AlleleID': 'allele_id', 'VariationID':'variation_id','Chromosome':'chromosome', 'Start':'coords','ReferenceAlleleVCF':'ref','AlternateAlleleVCF':'alt','GeneSymbol':'gene_symbol','uni_AC':'uniprot_id','Uniprot_iso':'uniprot_iso_id', 'ClinicalSignificance':'pathogenicity','PhenotypeList':'phenotype'}, inplace=True)

# introduce the composite_id column:
clinvar['composite_id'] = ['_'.join([str(i) for i in [a, b, c, d]]) for a, b, c, d in zip(clinvar['chromosome'], clinvar['coords'], clinvar['ref'], clinvar['alt'])]

# clean up pathogenicity and phenotype columns:
dictionary = {'Uncertain significance':'uncertain', 'Pathogenic':'pathogenic', 'Conflicting interpretations of pathogenicity':'conflicting', 'Likely pathogenic':'pathogenic', 'Benign':'benign', 'Likely benign':'benign', 'Pathogenic/Likely pathogenic':'pathogenic', 'Benign/Likely benign':'benign'}
for k, v in dictionary.items():
    clinvar['pathogenicity'] = clinvar['pathogenicity'].str.replace(k, v)
dictionary = {'pathogenic/pathogenic':'pathogenic', 'benign/benign':'benign'}
for k, v in dictionary.items():
    clinvar['pathogenicity'] = clinvar['pathogenicity'].str.replace(k, v)
clinvar['phenotype']  =[i.lower().replace('|', ' | ').replace('-', '').replace(' | not provided', '').replace(' | not specified', '').replace('not provided | ', '').replace('not specified | ', '').replace('not provided, ', '').replace(', not provided', '') for i in clinvar['phenotype']]
clinvar['phenotype'] = clinvar['phenotype'].replace({'not provided':None}) #change to proper None values
clinvar['phenotype'] = clinvar['phenotype'].replace({'not specified':None}) #change to proper None values
# filter out Ter mutations: 
clinvar = clinvar[~(clinvar['aa_change'].str.contains('Ter'))]
# reorder the columns:
clinvar = clinvar[clinvar.columns.tolist()[:6] + ['composite_id'] + clinvar.columns.tolist()[6:-1]]
clinvar.drop_duplicates(inplace=True)
clinvar.reset_index(inplace=True, drop=True)

# Now, we did compare reference amino acid to swissprot sequences but we did not compare to John's version, so I will do that:

#I need to prepare clinvar_m dataset first by introducing some columns
clinvar_m = clinvar.copy(deep=True)
clinvar_m.reset_index(inplace=True, drop=True)
clinvar_m['aa_ref'] = [i[:3] for i in clinvar_m['aa_change']]
clinvar_m['aa_alt'] = [i[-3:] for i in clinvar_m['aa_change']]
clinvar_m['aa_pos'] = [int(''.join([a for a in i if a.isdigit()])) for i in clinvar_m['aa_change']]
clinvar_m['aa_ref_short'] = clinvar_m['aa_ref'] 
clinvar_m['aa_alt_short'] = clinvar_m['aa_alt']
AA_dict = {'Phe':'F','Leu':'L','Ser':'S','Tyr':'Y','Cys':'C','Trp':'W','Pro':'P','His':'H','Gln':'Q','Arg':'R','Ile':'I','Met':'M','Thr':'T','Asn':'N','Lys':'K','Val':'V','Ala':'A','Asp':'D','Glu':'E','Gly':'G','Ter':'*'}
clinvar_m['aa_ref_short'] = [AA_dict.get(i) for i in clinvar_m['aa_ref_short']]
clinvar_m['aa_alt_short'] = [AA_dict.get(i) for i in clinvar_m['aa_alt_short']]

# loading John's dataset
query = '''select * from chopyan_db.uniprot_id_sequence as df'''
john_seq = pd.read_sql_query(query, con=eng)

compare_ref_aa = clinvar_m.merge(john_seq, how='inner', on='uniprot_id')
compare_ref_aa['length'] = [len(i) for i in compare_ref_aa['sequence'].values]

print(len(compare_ref_aa[compare_ref_aa['aa_pos'] > compare_ref_aa['length']]))
compare_ref_aa = compare_ref_aa[compare_ref_aa['aa_pos'] <= compare_ref_aa['length']]

#compare the actual sequences:
compare_ref_aa['swiss_prot_ref_aa'] = [seq[pos-1] for seq, pos in zip(compare_ref_aa['sequence'].values, compare_ref_aa['aa_pos'].values)]
print(len(compare_ref_aa[compare_ref_aa['aa_ref_short'] != compare_ref_aa['swiss_prot_ref_aa']]))
compare_ref_aa = compare_ref_aa[compare_ref_aa['aa_ref_short'] == compare_ref_aa['swiss_prot_ref_aa']]

clinvar = compare_ref_aa.copy(deep=True)
clinvar.drop(columns=['sequence','length','swiss_prot_ref_aa'], inplace=True)
clinvar.reset_index(inplace=True, drop=True)


KeyError: "['AlleleID', 'VariationID', 'Chromosome', 'Start', 'ReferenceAlleleVCF', 'AlternateAlleleVCF', 'GeneSymbol', 'uni_AC', 'ClinicalSignificance', 'PhenotypeList'] not in index"

In [194]:
clinvar_m = clinvar.copy(deep=True)

In [206]:
# I think I also want to run NDD annotation on this again, because I did it a bit differently from Kristina:

### I already made it lowercase, removed -, changed | into | with spaces around it. I also checked NaN values and there are None in this column:

#preparing human phenotype ontology (HPO) list of NDD phenotypes and keyword search data
#load HPO
hpo_ndds = pd.read_excel(find_file('HPO_NDDs.xlsx'), header=0)

#lower string, replace '-', and '|', and create a single string:
hpo_ndds['DISEASE_NAME'] = [i.lower() for i in hpo_ndds['DISEASE_NAME']]
hpo_ndds['DISEASE_NAME'] = hpo_ndds['DISEASE_NAME'].str.replace('-', ' ')
hpo_string_data = ' | '.join([i for i in hpo_ndds['DISEASE_NAME']])
filpat = r',? ?(Autosomal|Somatic) ?(Dominant|Recessive)? ?\d*|,? ?((Type)? \d\w?|Type \w{2}) ?'
hpo = pd.DataFrame(hpo_string_data.split(' | '))
hpo.rename(columns={0:'phenotype'}, inplace=True)
hpo.phenotype = hpo.phenotype.str.replace(filpat, '', flags = re.IGNORECASE, regex = True)
hpo_string_data = ' | '.join([i for i in hpo['phenotype']])

#prepare keyword search string:
keyword_search = ' | '.join(['neurodevelopment', 'learning disability', 'cognitive impairment', 'mental retardation', 'intellectual disability', 'autism', 'autistic', 'epilepsy', 'epileptic', 'developmental delay', 'delayed development'])
#learning disability is a synonym of intellectual disability, but for example learning difficulties can be referring to something else, like dyslexia
#cognitive impairment is also a synonym of intellectual disability
#I put neurodevelopment instead of neuro because neuro will also include neuropathies and so on

clinvar_m['hpo'] = clinvar_m['phenotype'] #had to do this because the ndd_annotation function is written for hpo and phenotype columns

def ndd_annotation(dataset, comparison_string, string_name):
    '''
    Annotate phenotypes as NDD associated or not, based on a list of NDD phenotypes
    dataset -> df; the dataset which contains 'phenotype' column which lists phenotypes associated with each variant, and 'hpo' column which lists hpo identifiers for those phenotypes
    comparison_string -> str; string which contains a list of NDD phenotypes separated with a |; I used either a list of NDD phenotypes provided by the HPO ontology or the keyword search list I built myself
    string_name -> str; name of the comparison_string, so that I can change the values in the corresponding columns
    
    Returns a dataframe with 3 additional columns: one which lists phenotypes which were found to be NDD-associated, second which lists their corresponding HPO identifiers and the last one which says True
    if at least one of the phenotypes in at least one of the patients were found to be NDD-associated. Also, I did the string matching both ways - if the phenotype in the phenotype column is in the
    string and also if elements of the string are matching to one of the phenotypes in the phenotype column. Either way, the phenotypes which were found to be a match are listed in the f'phenotype_ndd_in_{string_name}'
    column and those are the phenotypes that are in the phenotype column (they are not taken from the string).
    '''
    
    dataset[f'phenotype_ndd_in_{string_name}'] = False
    dataset[f'hpo_ndd_in_{string_name}'] = False
    dataset[f'ndd_in_{string_name}'] = False
    
    dataset[f'phenotype_ndd_in_{string_name}'] = dataset[f'phenotype_ndd_in_{string_name}'].astype(dtype='object') 
    dataset[f'hpo_ndd_in_{string_name}'] = dataset[f'phenotype_ndd_in_{string_name}'].astype(dtype='object')
    dataset[f'ndd_in_{string_name}'] = dataset[f'phenotype_ndd_in_{string_name}'].astype(dtype='object')
    
    for n, (phe, hpo) in enumerate(zip(dataset['phenotype'].values, dataset['hpo'].values)):
        if n % 10000 == 0:
            print('a 10 000 done')
        if type(phe) != type(hpo): #I expect them to be same type and if this is not the case, I want to be alarmed
            print('alarm')
        if dataset.loc[n, 'phenotype'] == None:
            dataset.at[n, f'phenotype_ndd_in_{string_name}'] = None
            dataset.at[n, f'hpo_ndd_in_{string_name}'] = None #and the third column will say false!
        comparison_list = list(set(comparison_string.split(' | '))) # hpo string may have some duplicates because it's basically a tree!
        if type(dataset.at[n, 'phenotype']) == str:
            phe_list = phe.split(' | ') #this is now a list of phenotypes
            hpo_list = hpo.split(' | ') #I defined it here because I will use it in the first and the second if statement
            if True in [True for i in phe_list if i in comparison_string]:
                dataset.at[n, f'ndd_in_{string_name}'] = True
                dataset.at[n, f'phenotype_ndd_in_{string_name}'] = ' | '.join([i for i in phe_list if i in comparison_string]) #if any of them are in the list
                get_indices = [phe_list.index(i) for i in phe_list if i in comparison_string]
                dataset.at[n, f'hpo_ndd_in_{string_name}'] = ' | '.join(list(np.array(hpo_list)[get_indices])) #get those exact HPO IDs also !
            ######################
            #now I want to check the other way around
            if True in [True for i in comparison_list if i in phe]:
                dataset.at[n, f'ndd_in_{string_name}'] = True #again, immediately set to True, so if it's either the previous thing or this thing, I want it to be set to true
                if True not in [True for i in phe_list if i in comparison_string]: #if the previous statement is not true, then there is no need to concatenate the 2 strings
                    #first let's get a list of matches from the comparison_list
                    list_of_matches = [i for i in comparison_list if i in phe]
                    #now let's get phenotypes which contain these matches!
                    dataset.at[n, f'phenotype_ndd_in_{string_name}'] = ' | '.join(list(OrderedDict.fromkeys([i for i in phe_list for x in list_of_matches if x in i]))) #if one of the elements contains both of the matches, it will be mentioned twice and there is no need for that; for example 'autistic behavior, intellectual disability | skeletal disorder', the first element will appear twice, but this is unlikely #used set
                    get_indices = list(OrderedDict.fromkeys([phe_list.index(i) for i in phe_list for x in list_of_matches if x in i])) #the reason I used ordereddict is because it preserves order!
                    dataset.at[n, f'hpo_ndd_in_{string_name}'] = ' | '.join(list(OrderedDict.fromkeys(list(np.array(hpo_list)[get_indices])))) #get those exact HPO IDs also ! #used set
                if True in [True for i in phe_list if i in comparison_string]: #if this is a True statement, then I want to concatenate this and the previous value
                    phe_value_1 = ' | '.join([i for i in phe_list if i in comparison_string])
                    get_indices = [phe_list.index(i) for i in phe_list if i in comparison_string]
                    hpo_value_1 = ' | '.join(list(np.array(hpo_list)[get_indices]))
                    #value 2:
                    list_of_matches = [i for i in comparison_list if i in phe]
                    phe_value_2 = ' | '.join(list(OrderedDict.fromkeys([i for i in phe_list for x in list_of_matches if x in i])))
                    get_indices = list(OrderedDict.fromkeys([phe_list.index(i) for i in phe_list for x in list_of_matches if x in i]))
                    hpo_value_2 = ' | '.join(list(OrderedDict.fromkeys(list(np.array(hpo_list)[get_indices]))))
                    #final value:
                    phe_value = phe_value_1 + ' | ' + phe_value_2
                    phe_value = ' | '.join(list(OrderedDict.fromkeys(phe_value.split(' | ')))) #I wanna take a set in that case #I'm afraid this might change the order but let's see, as long as we do the same thing with hpo, should be fine?
                    hpo_value = hpo_value_1 + ' | ' + hpo_value_2
                    hpo_value = ' | '.join(list(OrderedDict.fromkeys(hpo_value.split(' | '))))
                    dataset.at[n, f'phenotype_ndd_in_{string_name}'] = phe_value
                    dataset.at[n, f'hpo_ndd_in_{string_name}'] = hpo_value
            if (True not in [True for i in phe_list if i in comparison_string]) & (True not in [True for i in comparison_list if i in phe]):
                dataset.at[n, f'phenotype_ndd_in_{string_name}'] = None #if we cannot find matches either way, then we set this to None and the columns will stay 'False'
                dataset.at[n, f'hpo_ndd_in_{string_name}'] = None
        if type(dataset.at[n, 'phenotype']) == list: #this would be a list of strings
            new_value_phe = []
            new_value_hpo = []
            for phe_string, hpo_string in zip(phe, hpo): #we go through one string first
                if phe_string == None: #if one of the values is None, which means that the phenotype for one patient is not available and then hpo will also be None
                    pass
                else: #this is now if it's a string basically
                    phe_list = phe_string.split(' | ') #I gotta define this now for every string of the list, if type(dataset.at[n, 'phenotype']) == list
                    hpo_list = hpo_string.split(' | ')
                    if True in [True for i in phe_list if i in comparison_string]:
                        dataset.at[n, f'ndd_in_{string_name}'] = True
                        new_value_phe = new_value_phe + [i for i in phe_list if i in comparison_string] #I don't want to append a list, then it will be a list of lists
                        get_indices = [phe_list.index(i) for i in phe_list if i in comparison_string]
                        new_value_hpo = new_value_hpo + list(np.array(hpo_list)[get_indices])
                    ######################
                    #now I want to check the other way around
                    #I already defined the comparison list up there
                    if True in [True for i in comparison_list if i in phe_string]:
                        dataset.at[n, f'ndd_in_{string_name}'] = True #again, immediately set to True, so if it's either the previous thing or this thing, I want it to be set to true
                        #first let's get a list of matches from the comparison_list
                        list_of_matches = [i for i in comparison_list if i in phe_string]
                        #now let's get phenotypes which contain these matches!
                        new_value_phe = new_value_phe + list(OrderedDict.fromkeys([i for i in phe_list for x in list_of_matches if x in i]))
                        get_indices = list(OrderedDict.fromkeys([phe_list.index(i) for i in phe_list for x in list_of_matches if x in i]))
                        new_value_hpo = new_value_hpo + list(OrderedDict.fromkeys(list(np.array(hpo_list)[get_indices])))
                    if (True not in [True for i in phe_list if i in comparison_string]) & (True not in [True for i in comparison_list if i in phe_string]):
                        pass
            if new_value_phe == []: #if there are no None values and also if none of the phenotypes match
                new_value_phe = None
                new_value_hpo = None
            else:
                new_value_phe = ' | '.join(list(OrderedDict.fromkeys(new_value_phe))) #so far I was just concatenating strings to the new_value_hpo, but now i WANT
                new_value_hpo = ' | '.join(list(OrderedDict.fromkeys(new_value_hpo)))
            dataset.at[n, f'phenotype_ndd_in_{string_name}'] = new_value_phe
            dataset.at[n, f'hpo_ndd_in_{string_name}'] = new_value_hpo
    return dataset

clinvar_m = ndd_annotation(clinvar_m, hpo_string_data, 'hpo')
clinvar_m = ndd_annotation(clinvar_m, keyword_search, 'keyword_search')


a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 000 done
a 10 0

In [207]:
clinvar_m['ndd_phe'] = [1 if (a or b) == True else 0 for a,b in zip(clinvar_m['ndd_in_hpo'], clinvar_m['ndd_in_keyword_search'])]

# keep the columns I think are worth saving in a file:
# clinvar_m = clinvar_m[clinvar_m.columns.tolist()[:10] + ['aa_change', 'aa_ref','aa_alt', 'aa_pos', 'aa_ref_short', 'aa_alt_short', 'pathogenicity', 'phenotype', 'phenotype_ndd_in_hpo','ndd_in_hpo','phenotype_ndd_in_keyword_search','ndd_in_keyword_search','ndd_phe_m']]
# clinvar_m.to_csv('output/clinvar_filtered_kristina_milena.csv', header=True, sep='\t', index=False)
# clinvar_m = pd.read_csv(find_file('clinvar_filtered_kristina_milena.csv'), sep='\t', header=0)


In [210]:
new_clinvar = clinvar_m.copy(deep=True)

In [298]:
new_clinvar = new_clinvar[new_clinvar.columns.tolist()[:10] + ['aa_change', 'aa_ref','aa_alt', 'aa_pos', 'aa_ref_short', 'aa_alt_short', 'pathogenicity', 'phenotype', 'phenotype_ndd_in_hpo','ndd_in_hpo','phenotype_ndd_in_keyword_search','ndd_in_keyword_search','ndd_phe']]
%store new_clinvar
new_clinvar.to_csv('output/clinvar_filtered_milena_2023.csv', header=True, sep='\t', index=False)
new_clinvar = pd.read_csv(find_file('clinvar_filtered_milena_2023.csv'), sep='\t', header=0)
